# Tutorial for the OpenFermion-Dirac interface

The goal of this interface is to perform relativistic or non-relativistic calculation of 
chemical compounds through the Dirac program (see http://diracprogram.org)
in OpenFermion (https://github.com/quantumlib/OpenFermion). 
The resulting molecular Hamiltonian can then be encoded into a qubit Hamiltonian thanks to the Jordan-Wigner
transformation, for instance.

## Non relativistic calculation

First, define your atom or molecule. Here, the example of the hydrogen molecule is provided.

In [1]:
bond_length = 1.0
multiplicity = 1
charge = 0
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

Then, specify the basis set. For practical reason, the small STO-3G basis set is considered here. More complex basis sets can be used to perform Hartree-Fock (HF) or Coupled Cluster Single and Doubles (CCSD), but it becomes rapidly untractable for Full Configuration Interaction (FCI).

In [2]:
basis = 'STO-3G'

To know more about the valid basis sets, we refer the reader to the DIRAC documentation (http://diracprogram.org).

Given that the filename does not contain information about the bond length or the type of calculation, it is interesting to add it manually. We will perform here a CCSD calculation.

In [3]:
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

Now that every parameters are defined, one can construct the molecule thanks to the ```MolecularData_Dirac``` command

In [4]:
import os

data_directory=os.getcwd()

from openfermion_dirac import MolecularData_Dirac

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

Time to start the calculation ! First, defines which type of calculation you want to perform. The choices are SCF (Hartree-Fock) or CCSD. If you don't set any option, it will run an SCF calculation automatically. 
If you set ```run_ccsd = True```, it will perform a CCSD calculation. Performing CCSD also gives you access to the second-order Moller-Plesset perturbation (MP2) and the Hartree-Fock (HF) energy.

In [5]:
run_ccsd = True

Note that the non relativistic calculation uses the Levy-Leblond Hamiltonian. For some applications (outside the scope of this tutorial) one may want to use the NONREL Hamiltonian instead. This can be set with ```NONREL=True``` in ```run_dirac```.

Several files are generated during the calculation: 
- The input file (.inp)
- The geometry file (.xyz)
- The output file (.out)
- The integral files (MRCONEE and MDCINT)
- The FCIDUMP file
- Optionally, an hdf5 file which saves several informations (as shown latter on).

You can define all the files that you want to keep or to discard at the end of the calculation. Note that the geometry of the molecule and the input file are included in the Dirac output file, such that only this file can be kept.
The MRCONEE and MDCINT contain the one- and two-electron integrals, respectively. The ```dirac_openfermion_mointegral_export.x``` transforms those files into a FCIDUMP file containing all the molecular integrals. Only the latter file is then needed to construct the molecular Hamiltonian in OpenFermion. This molecular Hamiltonian can then be transformed into a Qubit Hamiltonian, as shown in the following.

In [6]:
delete_input = True
delete_xyz = True
delete_output = False
delete_MRCONEE = True
delete_MDCINT = True

Note that is fcidump=False and run_ccsd = False, the MRCONEE and MDCINT files will not be extracted (not needed).

To start the calculation, use the command ```run_dirac``` as follows:

In [7]:
from openfermion_dirac import run_dirac

fcidump=True

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

Note that ```fcidump=True``` has to be set if you want to extract the second-quantized molecular Hamiltonian in OpenFermion from the object ```molecule```:

In [8]:
molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
print(molecular_hamiltonian)

() 0.5291772083
((0, 1), (0, 0)) -1.11084417705
((1, 1), (1, 0)) -1.11084417705
((2, 1), (2, 0)) -0.5891210130952
((3, 1), (3, 0)) -0.5891210130952
((0, 1), (0, 1), (0, 0), (0, 0)) 0.313201246336
((0, 1), (0, 1), (2, 0), (2, 0)) 0.0983952893027
((0, 1), (1, 1), (1, 0), (0, 0)) 0.313201246336
((0, 1), (1, 1), (3, 0), (2, 0)) 0.0983952893027
((0, 1), (2, 1), (0, 0), (2, 0)) 0.0983952893027
((0, 1), (2, 1), (2, 0), (0, 0)) 0.3108533766756
((0, 1), (3, 1), (1, 0), (2, 0)) 0.0983952893027
((0, 1), (3, 1), (3, 0), (0, 0)) 0.3108533766756
((1, 1), (0, 1), (0, 0), (1, 0)) 0.313201246336
((1, 1), (0, 1), (2, 0), (3, 0)) 0.0983952893027
((1, 1), (1, 1), (1, 0), (1, 0)) 0.313201246336
((1, 1), (1, 1), (3, 0), (3, 0)) 0.0983952893027
((1, 1), (2, 1), (0, 0), (3, 0)) 0.0983952893027
((1, 1), (2, 1), (2, 0), (1, 0)) 0.3108533766756
((1, 1), (3, 1), (1, 0), (3, 0)) 0.0983952893027
((1, 1), (3, 1), (3, 0), (1, 0)) 0.3108533766756
((2, 1), (0, 1), (0, 0), (2, 0)) 0.3108533766756
((2, 1), (0, 1), (2, 0)

and encode it into the qubit Hamiltonian thanks to the Jordan-Wigner transformation, provided by OpenFermion:

In [9]:
from openfermion.transforms import jordan_wigner

qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
print(qubit_hamiltonian)

-0.32760821099250037 [] +
-0.04919764465135 [X0 X1 Y2 Y3] +
0.04919764465135 [X0 Y1 Y2 X3] +
0.04919764465135 [Y0 X1 X2 Y3] +
-0.04919764465135 [Y0 Y1 X2 X3] +
0.13716573333275 [Z0] +
0.156600623168 [Z0 Z1] +
0.10622904368644998 [Z0 Z2] +
0.1554266883378 [Z0 Z3] +
0.13716573333275 [Z1] +
0.1554266883378 [Z1 Z2] +
0.10622904368644998 [Z1 Z3] +
-0.13036290911284995 [Z2] +
0.1632676836362 [Z2 Z3] +
-0.13036290911284995 [Z3]


The energies calculated from the Dirac program are extracted as follows:

In [10]:
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Hartree-Fock energy of -1.0661086531285879 Hartree.
MP2 energy of -1.086665367911370 Hartree.
CCSD energy of -1.101150333280237 Hartree.


Note that it is usual to set consider the point nucleus model instead of the Gaussian charge distribution (default)
in a non-relativistic calculation. This can be set by specifying:

In [11]:
point_nucleus = True

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    point_nucleus=point_nucleus)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Hartree-Fock energy of -1.0661086541045273 Hartree.
MP2 energy of -1.086665368896307 Hartree.
CCSD energy of -1.101150334281559 Hartree.


We can observe small changes in the energies. Then one can build the qubit Hamiltonian, which operates in the Fock space (i.e., independently of the number of particle) and is solved
in OpenFermion with the eigenspectrum function:

In [12]:
from openfermion.utils import eigenspectrum

evs = eigenspectrum(qubit_hamiltonian)
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.10115033 -0.74587181 -0.74587181 -0.74587181 -0.60860674 -0.60860674
 -0.58166697 -0.58166697 -0.35229065 -0.06021533 -0.06021533 -0.0599438
 -0.0599438   0.0390476   0.50196591  0.52917721]


## FCI Calculation

The FCI ground-state energy can also be calculated as follows (let us take the example of LiH, for which CCSD is not exact anymore)

In [13]:
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., bond_length))]
run_fci = True
description = 'R' + str(bond_length) + '_fci'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule_fci = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_fci=run_fci)

# Let us compare with the CCSD energy:
run_ccsd = True
description = 'R' + str(bond_length) + '_ccsd'
molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule_ccsd = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

print('Hartree-Fock energy of {} Hartree.'.format(molecule_ccsd.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule_ccsd.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule_ccsd.get_energies()[2]))
print('FCI energy of {} Hartree.'.format(molecule_fci.get_energies()[3]))

/Users/bsenjean/Documents/Git_repositories/Openfermion-Dirac/openfermion_dirac/_run_dirac.py:130: Warning: run_fci is simply implemented to get the FCI ground-state energy for now. Using it to get anything else is highly not recommended. This option has not been tested meticulously and it might lead to errors depending on the system that you want to treat.
  Warning)


Hartree-Fock energy of -7.7673621382862628 Hartree.
MP2 energy of -7.778799092752027 Hartree.
CCSD energy of -7.784454824479287 Hartree.
FCI energy of -7.784460282090 Hartree.


## DFT Calculation

Density-functional theory is one of the most widely used approach in quantum chemistry due to its mean-field-like formulation, thus leading to a much cheaper computational cost in comparison to multiconfigurational wavefunction methods. In Dirac program, several functionals are available (see http://www.diracprogram.org/doc/master/manual/dftcfun.html?highlight=dft) and can be called in this interface by specifying run_dirac="X", where X is the choosen approximate exchange-correlation functional.

In [14]:
run_dft="LDA"
description = 'R' + str(bond_length) + '_' + run_dft

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=True,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    run_dft=run_dft)

print('DFT energy of {} Hartree.'.format(molecule.get_energies()[0]))

DFT energy of -7.6988055970620017 Hartree.


## Relativistic Calculation

To perform a relativistic calculation by considering the full Dirac-Coulomb Hamiltonian, a simple option has to be switched on. Note that one should not switch on the ```point_nucleus``` option for relativistic calculation.

In [15]:
relativistic = True
basis = "STO-3G"
bond_length = 1.0
multiplicity = 1
charge = 0
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

run_ccsd = False
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               relativistic=relativistic,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    relativistic=relativistic)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Hartree-Fock energy of -1.0661226627178420 Hartree.
MP2 energy of None Hartree.
CCSD energy of None Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.1011657  -0.74591589 -0.74591589 -0.74591589 -0.60865735 -0.60865735
 -0.58167347 -0.58167347 -0.35233876 -0.06029881 -0.06029881 -0.05998437
 -0.05998437  0.03896802  0.50187888  0.52917721]


Small differences are seen between the relativistic energies and the non-relativistic ones obtained previously.
Of course, the difference is quite small for the hydrogen molecule but it is known to be very important especially for heavy elements.

### Extraction of the CC amplitudes

Not so much tested for now. I advise to keep ```relativistic=True```, as well as ```symmetry=False```.
The format has been chosen to be the same as in the other interfaces with OpenFermion (like PySCF).

In [23]:
ccamp = True
run_ccsd = True
relativistic = True
symmetry = False
if run_ccsd:
    description = 'ccsd'
else:
    description = 'scf'

geometry = [('C', (0., 0., 0.))]

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               relativistic=relativistic,
                               symmetry=symmetry,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    relativistic=relativistic,
                    ccamp=ccamp,
                    symmetry=symmetry)

Then one can get rid of all zeros amplitudes. That can be useful to reduce the number of terms in the Uniraty Coupled Cluster ansatz in OpenFermion for instance.

In [24]:
def _cut_ccsd_amplitudes(single_amplitudes, double_amplitudes, threshold):
    if not (isinstance(single_amplitudes, list) or isinstance(double_amplitudes, list)):
        raise TypeError('Input amplitudes must be a list.')
    sing_amp = single_amplitudes[:]
    doub_amp = double_amplitudes[:]
    list_sing_amp_del = []
    list_doub_amp_del = []
    for i in range(len(single_amplitudes)):
        if (abs(single_amplitudes[i][1]) < threshold):
            list_sing_amp_del.append(i)
    for i in range(len(double_amplitudes)):     
        if (abs(double_amplitudes[i][1]) < threshold):
            list_doub_amp_del.append(i)
    # Get rid of all zeros:
    for i in sorted(list_sing_amp_del, reverse=True):
        del sing_amp[i]
    for i in sorted(list_doub_amp_del, reverse=True):
        del doub_amp[i]
    return sing_amp, doub_amp

from openfermion import *
from scipy.sparse.linalg import expm_multiply


# TRUNCATE AMPLITUDES:
threshold = 1e-8
sing_amps, doub_amps = uccsd_convert_amplitude_format(molecule.get_ccamps()[0], molecule.get_ccamps()[1])
sing_amps_trunc, doub_amps_trunc = _cut_ccsd_amplitudes(sing_amps,doub_amps,threshold)
print("CCSD single amps: {}".format(sing_amps_trunc))
print("CCSD double amps: {}".format(doub_amps_trunc))

CCSD single amps: [[[7, 2], (-9.529791772281856e-09-5.644778301946054e-09j)], [[9, 5], (-9.529790385169902e-09+5.644775091431833e-09j)]]
CCSD double amps: [[[6, 0, 8, 3], (-0.0001715867318030119+3.602076021644978e-18j)], [[6, 0, 8, 4], (-0.0006260662279481931-4.618293779486494e-15j)], [[6, 1, 8, 3], (-0.0006260662279481861+4.650144269740836e-15j)], [[6, 1, 8, 4], (-0.040407097026408545+7.231741952722165e-16j)], [[6, 2, 8, 5], (-0.21874636759737953-4.738876195394322e-15j)], [[7, 0, 9, 3], (-0.00017158673531763712-1.9787844132240768e-18j)], [[7, 0, 9, 4], (-0.0006260662411235532+5.01895343271201e-15j)], [[7, 1, 9, 3], (-0.0006260662411235532-5.028947108576946e-15j)], [[7, 1, 9, 4], (-0.04040709792732233-7.30164382444635e-16j)], [[7, 2, 9, 5], (-0.21874636162395333+4.731428201497912e-15j)], [[8, 3, 6, 0], (-0.0001715867318030119+3.602076021644978e-18j)], [[8, 3, 6, 1], (-0.0006260662279481861+4.650144269740836e-15j)], [[8, 4, 6, 0], (-0.0006260662279481931-4.618293779486494e-15j)], [[8, 4

### Change the speed of light

One can play with the speed of light (137 a.u.) in order to increase the relativistic effects:

In [25]:
speed_of_light = 3.0
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]

run_ccsd = False
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               speed_of_light=speed_of_light,
                               description=description,
                               relativistic=relativistic,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    relativistic=relativistic,
                    speed_of_light=speed_of_light)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))

Hartree-Fock energy of -1.0942741633461521 Hartree.
MP2 energy of None Hartree.
CCSD energy of None Hartree.
Solving the Qubit Hamiltonian (Jordan-Wigner): 
 [-1.13213496 -0.82925112 -0.82925112 -0.82925112 -0.70678081 -0.70678081
 -0.59452925 -0.59452925 -0.44310418 -0.21989424 -0.21989424 -0.13468642
 -0.13468642 -0.10968398  0.33171782  0.52917721]


The change in energy is now much more important, showing the significance of considering relativistic effects in this case.

## Active space option

Let us take the example of the lithium hydride molecule LiH:

In [33]:
basis = 'STO-3G'
bond_length = 2.0
multiplicity = 1
charge = 0

delete_input = True
delete_xyz = True
delete_output = False
delete_MRCONEE = True
delete_MDCINT = True
delete_FCIDUMP = False
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., bond_length))]
description = "R" + str(bond_length) + "_scf"

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

For LiH, let's first do a calculation to have information on the orbital energies. To get the orbitals, we only need the SCF calculation which is the default (i.e., switch-off ```run_ccsd```).

In [34]:
import sys
save = True
point_nucleus=True

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    save=save)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

# This could be done to check if the saving file exist already or not.
#if os.path.exists("{}/{}.hdf5".format(data_directory,molecule.name)) is False:
if os.path.exists("{}.hdf5".format(molecule.filename)) is False:
    print("No file found. You should first run a calculation with save=True (see LiH_save.py)")
    sys.exit(0)

print("The spin-orbitals and their associated energy:{}\n".format(molecule.get_from_file("orbital_energies")))

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print(evs)

CCAMP not found, set run_ccsd and ccamp to True if needed.
Hartree-Fock energy of -7.8309055873879219 Hartree.
The spin-orbitals and their associated energy:{1: -2.361187885634, 3: -0.2501066514045, 5: 0.07327904161477, 7: 0.1621056645776, 9: 0.1621056645776, 11: 0.4326451334071, 2: -2.361187885634, 4: -0.2501066514045, 6: 0.07327904161477, 8: 0.1621056645776, 10: 0.1621056645776, 12: 0.4326451334071}

[-7.86108777 -7.78953175 -7.78953175 ...  0.79376581  1.28714688
  1.61687514]


LiH is a system with 4 electrons. We see that the first spatial orbital (indexed by 1 and 2) has an energy much lower than the second one (indexed by 3 and 4). Consequently, one can freeze the two electrons in this orbital and consider only the 2 other electrons as active. We can also get rid of the last spatial orbital with the highest energy. By doing so, only 8 spin-orbitals with 2 electrons will form our active space.

In [38]:
active = [2,3,4,5] # denote spatial orbitals

run_ccsd=True

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    active=active)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('CASCI energy of {} Hartree'.format(evs[0]))

# Compare with FCI:
molecule = run_dirac(molecule,
                    fcidump=False,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_fci=run_fci)

FCI_energy = molecule.get_energies()[3]

# Compare with FCI within the active space (in other words, CASCI)
molecule = run_dirac(molecule,
                    fcidump=False,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    active=active,
                    run_fci=run_fci)

CASCI_energy = molecule.get_energies()[3]

print('CASCI energy of {} Hartree'.format(CASCI_energy))
print('FCI energy of {} Hartree'.format(FCI_energy))

Hartree-Fock energy of -7.8309055873879219 Hartree.
MP2 energy of -7.832575301374425 Hartree.
CCSD energy of -7.832886005802561 Hartree.
CASCI energy of -7.832886005803047 Hartree
run_fci and active keywords are not False. Note that this will run a CASCI and not a FCI.
CASCI energy of -7.832886005803 Hartree
FCI energy of -7.861087774775 Hartree


The calculation is much faster than if all the spin-orbitals are considered active, at the expense of a lower accuracy.

In [39]:
print("\nLet's add the last virtual orbitals:\n")
active = [2,3,4,5,6]

run_ccsd=True

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    active=active)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Ground-state energy of {} Hartree'.format(evs[0]))

print("\nLet's do something crazy:\n")
active = [1,3,5,6]

run_ccsd=True

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    active=active)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Ground-state energy of {} Hartree'.format(evs[0]))


Let's add the last virtual orbitals:

Hartree-Fock energy of -7.8309055873879219 Hartree.
MP2 energy of -7.848142751635732 Hartree.
CCSD energy of -7.860828260528564 Hartree.
Ground-state energy of -7.8608282605290185 Hartree

Let's do something crazy:

Hartree-Fock energy of -7.8309055873879219 Hartree.
MP2 energy of -7.831051912931489 Hartree.
CCSD energy of -7.831037639406328 Hartree.
Ground-state energy of -7.831037639406415 Hartree


One can see that MP2 and CCSD energies are also changing, because only the amplitudes of active orbitals will be considered.

# Other more specific options

## Getting other important files

DIRAC calculation can output several interesting files that are not written if not asked explicitly. For instance, OpenFermion-Dirac ask for the extraction of the ```MRCONEE``` and ```MDCINT``` file by default, which are unformatted files containing the electronic integrals, then transformed into a formatted FCIDUMP file. ```MDPROP``` is also extracted automatically if ```propint``` is ```True```. Another file that can be of interest is the ```DFCOEF``` one which contains all the molecular orbital coefficients, and information on the basis set (basis functiosn, their exponents and coefficients, etc.). This file is necessary to extract if you want to restart a calculation. Let's do a test with an SCF calculation that can last long (but not too long for the sake of the tutorial, it should last around 30 secondes): benzene in the cc-pVDZ basis set. (It should be noted that I am only doing an SCF calculation and I don't need the ```FCIDUMP``` file, such that the 4-index transformation can be omitted. Otherwise the calculation takes much longer.)

In [ ]:
charge       = 0
multiplicity = 1
basis        = "cc-pVDZ"
description  = ""
get          = "DFCOEF"
geometry = [
    ('C',(      -1.39880212,       0.00000000,       0.00000000)),
    ('C',(       1.39880212,       0.00000000,       0.00000000)),
    ('C',(      -0.69940106,      -1.21139817,       0.00000000)),
    ('C',(      -0.69940106,       1.21139817,       0.00000000)),
    ('C',(       0.69940106,       1.21139817,       0.00000000)),
    ('C',(       0.69940106,      -1.21139817,       0.00000000)),
    ('H',(      -2.49009098,       0.00000000,       0.00000000)),
    ('H',(      -1.24504549,       2.15648204,       0.00000000)),
    ('H',(       1.24504549,       2.15648204,       0.00000000)),
    ('H',(       2.49009098,       0.00000000,       0.00000000)),
    ('H',(       1.24504549,      -2.15648204,       0.00000000)),
    ('H',(      -1.24504549,      -2.15648204,       0.00000000))]

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    get=get,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

Then, let's restart the calculation with the generated DFCOEF file. Given that the MOs are already the Hartree-Fock ones, the SCF calculation should converge in one iteration and the calculation will be much faster.

In [ ]:
restart = True
molecule = run_dirac(molecule,
                    restart=restart,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

## Molecular Properties

Different properties can be computed within the Dirac program.
One can ask for properties calculation by creating a list ```properties = []``` containing keywords corresponding to the given properties of interest.

In [ ]:
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
basis = "STO-3G"
properties = ['MOLGRD','DIPOLE','QUADRUPOLE','EFG','POLARIZABILITY'] 
delete_output = False
molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    properties=properties,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

print('Electric dipole moment: {}'.format(molecule.get_elecdipole()))
print('Electric quadrupole moment: {}'.format(molecule.get_elecquadrupole()))
print('Electric (dipole) polarizability: {}'.format(molecule.get_elecpolarizability()))

For now, the electric dipole, quadrupole and polarizability can be extracted. For other properties, one has to look directly at the output.

One can also extract the PROPINT file that contains all < phi_i | O | phi_j >, i.e. the property integrals in the MO basis for a given one-electron operator O. This can be done using the ```propint``` keyword as follows:

In [ ]:
geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., 1.0))]
basis = "STO-3G"
propint = "ZDIPLEN"
# another formatted file to construct the unformatted PROPINT can be deleted:
delete_MDPROP = True
properties = ['DIPOLE']
relativisitc = False


molecule = MolecularData_Dirac(geometry=geometry,
                    basis=basis,
                    multiplicity=multiplicity,
                    charge=charge,
                    description=description,
                    relativistic=relativistic,
                    data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    propint=propint,
                    properties=properties,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    delete_MDPROP=delete_MDPROP,
                    relativistic=relativistic,
                    run_ccsd=run_ccsd)

property_hamiltonian = molecule.get_property_hamiltonian()
print(property_hamiltonian)


# Check if we indeed get the correct dipole moment. The dipole moment using the "properties" keywords gives:
print('Approximate Electric dipole moment (from Dirac program): {}'.format(molecule.get_elecdipole()))

# Compare with - < Psi_0 | z | Psi_0 > where Psi_0 is the FCI ground-state, i.e. giving the FCI dipole moment.
qubit_hamiltonian = jordan_wigner(molecule.get_molecular_hamiltonian()[0])
# Compute the states. vecs[:,0] is the ground state.
import numpy as np
from openfermion.transforms import get_sparse_operator
qubit_hamiltonian = get_sparse_operator(qubit_hamiltonian).todense()
evs, vecs = np.linalg.eigh(qubit_hamiltonian)

# Compute the expectation value of the first derivative Hamiltonians. It should give the dipole moment (up to a minus sign)
property_ham = jordan_wigner(property_hamiltonian)
property_ham = get_sparse_operator(property_ham).todense()
zdip_analytic = vecs[:,0].conj().T @ property_ham @ vecs[:,0]
print('FCI Electric dipole moment: {}'.format(- zdip_analytic))

## Add one-electron operators to the Hamiltonian

You can add a finite field in the x,y or z direction for instance. Other one-electron operators can be added. The field strength of the operator is specified with COMFACTOR.
For more informations, see Dirac's documentation.
Here, I describe how you can add a finite electric field in the z direction, for the H2 molecule.

In [ ]:
# Set molecule parameters.
# a ghost nuclei "foo" is added to lower the symmetry.
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length)), ('foo', (0.,0., 10.))]
# The basis has to be changed so that the nuclei "foo" is indeed treated as a ghost nuclei.
basis = 'special'
special_basis = ["STO-3G","foo NOBASIS"]
# Define the operator: [[op1,keywords,strength],[op2,keywords,strength],[op3,...],[...]]
operator = [["ZDIPLEN","COMFACTOR",0.001]]

run_ccsd = True
point_nucleus = True
description = 'R' + str(bond_length) + '_ccsd'

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               special_basis=special_basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    operator=operator,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()

This has performed a finite perturbation calculation with the z dipole length operator added to the Hamiltonian: H + 0.001 z

We can actually use this to compute the dipole moment by finite difference. To do so, just compute the "property_hamiltonian" described above by finite difference (H(+zdip/2) - H(-zdip/2))/zdip, where H(zdip) is the molecular Hamiltonian obtained above. By performing the same calculation as before to get the FCI dipole moment, you should find the same result than without finite difference (with ```propint```).

## Save option

Let us go back in a non relativistic framework (although everything that follows can be done with the ```relativistic = True``` option).
More options can be set in the calculation. The first one is the ```save``` option that allows you to save your calculation in the HDF5 format. Note that ```fcidump``` has to be equal to ```True```. The ouput can be deleted, but I advise not to do it as it contains much more information on the performed calculation.

In [ ]:
save = True
basis = "STO-3G"
geometry = [('H', (0., 0., 0.)), ('H', (0., 0., bond_length))]
point_nucleus = True
properties = ['MOLGRD','DIPOLE','QUADRUPOLE','EFG','POLARIZABILITY']

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=fcidump,
                    point_nucleus=point_nucleus,
                    properties=properties,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd,
                    save=save)

Then check is the HDF5 file has been created

In [ ]:
import sys
#if os.path.exists("{}/{}.hdf5".format(data_directory,molecule.name)) is False:
if os.path.exists("{}.hdf5".format(molecule.filename)) is False:
      sys.exit(0)
print("HDF5 file found, loading from file. Name : {}".format(molecule.get_from_file('name')))

You can extract the following information from this file thanks to the function ```get_from_file()```:

In [ ]:
print('Name : {}'.format(molecule.get_from_file('name')))
print('The description is {}.'.format(molecule.get_from_file('description')))
print('System with atoms {} and protons {}.'.format(molecule.get_from_file('atoms'),
                                    molecule.get_from_file('protons')))
print('System of {} atoms and {} electrons.'.format(molecule.get_from_file('n_atoms'),
                                                             molecule.get_from_file('n_electrons')))
print('With charge {} and multiplicity {}.'.format(molecule.get_from_file('charge'),
                                                  molecule.get_from_file('multiplicity')))
print('With basis-set {}.'.format(molecule.get_from_file('basis')))
print('With {} spin-orbitals with corresponding energies {}.'.format(molecule.get_from_file('n_orbitals'),
                                                                     molecule.get_from_file('orbital_energies')))
print('With {} qubits.'.format(molecule.get_from_file('n_qubits')))
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_from_file('hf_energy')))
print('MP2 energy of {} Hartree.'.format(molecule.get_from_file('mp2_energy')))
print('CCSD energy of {} Hartree.'.format(molecule.get_from_file('ccsd_energy')))

E_core=molecule.get_from_file('nuclear_repulsion')
one_body_coeff = molecule.get_from_file('one_body_coefficients')
two_body_coeff = molecule.get_from_file('two_body_coefficients')
molecular_ham_print = molecule.get_from_file('print_molecular_hamiltonian')
print('Core energy : {} Hartree.'.format(E_core))
print('Electric dipole moment: {}'.format(molecule.get_from_file('elec_dipole')))
print('Electric quadrupole moment: {}'.format(molecule.get_from_file('elec_quadrupole')))
print('Electric (dipole) polarizability: {}'.format(molecule.get_from_file('elec_polarizability')))
print('Molecular Hamiltonian : {}'.format(molecular_ham_print))

Note that this molecular Hamiltonian cannot be used directly to construct the qubit Hamiltonian. It has to be reconstruct first thanks to the core energy and the one- and two-body coefficients:

In [ ]:
from openfermion.ops import InteractionOperator

molecular_ham = InteractionOperator(float(E_core),one_body_coeff,two_body_coeff)
qubit_ham = jordan_wigner(molecular_ham)
evs = eigenspectrum(qubit_ham)
print("Solution of the qubit Hamiltonian :\n {}".format(evs))

## Average-of-configuration open-shell Hartree-Fock

http://www.diracprogram.org/doc/master/tutorials/open_shell_scf/aoc.html

Note that DIRAC does not have restricted open-shell Hartree-Fock implemented, as it is a relativistic code: spin-orbit interaction couples spin and spatial degrees of freedom and make the formalism much more complicated since one cannot exploit spin symmetry alone for fixing the expansion coefficients in the reference configuration state function (CSF) which serves as the trial function.
Instead of optimizing the energy for a single open-shell state, we shall optimize the energy for a limited set of open-shell states. See the link above for more information.

As an example, let's consider the carbon atom. The two valence electrons are spread in 6 pi spinors, and one can do the average of configuration to make spherically averaged orbitals.

List of 3 values, 
- the first one for the number of closed-shell electrons
- the second one for the number of open-shell
- the third is for the fractional occupation (#active electrons/#active spinor)

Let us consider the problem without symmetry, otherwise the setup of the openshell keywords is more complicated and one should refer the DIRAC documentation. By the same time, let's look at the influence of the symmetry on the HF energy.

In [ ]:
geometry = [('C', (0., 0., 0.))]

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    symmetry=True)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    symmetry=False)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

openshell = ["4","1","2/6"]
molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    openshell=openshell,
                    symmetry=False)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

## Uncontracted Basis-set

 If one add the keyword for uncontract basis set, the SCF energy gets even lower.

In [ ]:
uncontract = True
molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    uncontract=uncontract)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    symmetry=False,
                    uncontract=uncontract)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    symmetry=False,
                    openshell=openshell,
                    uncontract=uncontract)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))

## Special Basis-set

The last option I want to discuss is the special basis set. One can specify a given basis set for a specific element only.
An example is the Helium hydride ion HeH+, where the ```STO-3G``` basis can be considered for the Helium atom and the ```cc-pVDZ``` basis for the hydrogen atom.
To do so, set the basis-set to "special", and defines a special basis as a list of two string, as follows:

In [ ]:
geometry = [('He', (0., 0., 0.)), ('H', (0., 0., bond_length))]
charge = 1
multiplicity = 1
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'

# necessary keyword to specify that we are using a special basis
basis="special"
# list of two string : [default basis, special basis for a given atomic species]
special_basis=["STO-3G","H BASIS cc-pVDZ"]


molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               special_basis=special_basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    point_nucleus=point_nucleus,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

## Manual options

The Dirac program possesses a lot of different options that can be inserted manually in the script. The ```manual_option``` keyword can be used in this respect. Here, I described an option that was necessary to perform CCSD calculation on HeH+, due to a known bug that has now been fixed.

In [ ]:
run_ccsd = True
if run_ccsd:
    description = 'R' + str(bond_length) + '_ccsd'
else:
    description = 'R' + str(bond_length) + '_scf'
    
basis="STO-3G"
manual_option = """**RELCCSD
*CCENER
.NOSDT""" # or manual_option = "**RELCCSD\n*CCENER\n.NOSDT". This is used to switch off the triples correction.

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    fcidump=True,
                    point_nucleus=point_nucleus,
                    manual_option=manual_option,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))

Note that the qubit Hamiltonian operates in the Fock space. Hence, the ground-state is the ground-state is not always the expected one. This is especially true for ionic species, such as HeH+ for instance. One can use the symmetry conserving bravyi kitaev transformation to get only the states we are interested of:

In [ ]:
molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Solving the Qubit Hamiltonian (Jordan-Wigner): \n {}'.format(evs))


from openfermion.transforms import get_fermion_operator, symmetry_conserving_bravyi_kitaev

#symmetry_conserving_bravyi_kitaev(fermionicoperator,number_of_active_orbs,number_of_active_elec)

number_orbs = len(molecule.get_integrals_FCIDUMP()[1])
fermion_operator = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = symmetry_conserving_bravyi_kitaev(fermion_operator,number_orbs,2)
evs = eigenspectrum(qubit_hamiltonian)

print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Bravyi-Kitaev) Symmetry conserving: \n {}'.format(evs))

# Known actual issues 

From the side of Dirac with CCSD:

- Open-shell with CCSD: For Be+, HeH, or other open-shell systems, the starting Slater determinant for the CCSD calculation is not well determined. Because of this, CCSD calculation usually get rid of electrons and consider a closed-shell system (like Be2+ instead of Be+, or HeH+ instead of HeH). The repartition of the electrons in the determinant can be specified manually, but no automatic solution has been implemented yet.
Note that the FCI calculation may work, as shown in the following case of Beryllium.

### Open-Shell problems (Be+)

CCSD computes the energy of Be2+ instead of Be+. FCI is correct here.

In [ ]:
basis = 'sto-3g'
multiplicity = 2
charge = 1

geometry = [('Be', (0., 0., 0.))]

run_ccsd = 1
description = ''

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule_ccsd = run_dirac(molecule,
                    fcidump=fcidump,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_ccsd=run_ccsd)

molecular_hamiltonian = molecule.get_molecular_hamiltonian()[0]
qubit_hamiltonian = jordan_wigner(molecular_hamiltonian)
evs = eigenspectrum(qubit_hamiltonian)
print('Hartree-Fock energy of {} Hartree.'.format(molecule.get_energies()[0]))
print('MP2 energy of {} Hartree.'.format(molecule.get_energies()[1]))
print('CCSD energy of {} Hartree.'.format(molecule.get_energies()[2]))
print('Solving the Qubit Hamiltonian (Jordan-Wigner): first:{} ; twentieth:{} '.format(evs[0],evs[19])) 
# The 20th energy is the one of Be+. The first is the energy of Be.
number_orbs = len(molecule.get_integrals_FCIDUMP()[1])
fermion_operator = get_fermion_operator(molecular_hamiltonian)
qubit_hamiltonian = symmetry_conserving_bravyi_kitaev(fermion_operator,number_orbs,3)
evs = eigenspectrum(qubit_hamiltonian)
print('Solving the Qubit Hamiltonian (Bravyi-Kitaev) Symmetry conserving: \n \
      {}, {}, {}, {}'.format(evs[0],evs[1],evs[2],evs[3]))
print('For unknown reasons, a triplet state seems to be obtained here, \
while the ground-state of Be+ should be a doublet.')

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_fci=run_fci)

print('FCI energy of Be+: {} Hartree.'.format(molecule.get_energies()[3]))

multiplicity = 1
charge = 0

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_fci=run_fci)

print('FCI energy of Be: {} Hartree.'.format(molecule.get_energies()[3]))

multiplicity = 1
charge = 2

molecule = MolecularData_Dirac(geometry=geometry,
                               basis=basis,
                               multiplicity=multiplicity,
                               charge=charge,
                               description=description,
                               data_directory=data_directory)

molecule = run_dirac(molecule,
                    delete_input=delete_input,
                    delete_xyz=delete_xyz,
                    delete_output=delete_output,
                    delete_MRCONEE=delete_MRCONEE,
                    delete_MDCINT=delete_MDCINT,
                    run_fci=run_fci)

print('FCI energy of Be2+: {} Hartree.'.format(molecule.get_energies()[3]))